<a href="https://colab.research.google.com/github/daayvm/DiplomadoCD/blob/main/Tarea2_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Integrantes:

Centeno Amado Eduardo

Orozpe Frias Erick Adrian

Pineda Gaona Maricruz

Rodriguez Teodores Armando

Velasco Martell Dayra

Elabora un modelo de regresión utilizando cada uno de los kernels de SVR, donde tu variable objetivo es Rented Bike Count. Encuentra tu mejor modelo después del análisis de datos. Finalmente compara y concluye.

Bibliotecas

In [36]:
#General
!pip install skimpy
import numpy as np
import pandas as pd
from skimpy import skim

#Estadisticas
from scipy import stats

#Graficas
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from pandas.plotting import andrews_curves


#Valores atipicos
!pip install pyod
from pyod.models.mcd import MCD
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.mcd import MCD

#Modelo de regresion
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.svm import SVR # Agregado para importar SVR

#Guardar los modelos
import joblib
import pickle

Descripción de las Variables:

* Temperature (Temperatura): Temperatura del aire en grados Celsius (C).

* Humidity (Humedad): Humedad relativa del aire en porcentaje (%).

* Wind speed (Velocidad del Viento): Velocidad del viento en metros por segundo (m/s).

* Visibility (Visibilidad): Distancia de visibilidad en 10 metros (10m).

* Dew point temperature (Temperatura del Punto de Rocío): Temperatura del punto de rocío en grados Celsius (C).

* Solar Radiation (Radiación Solar): Intensidad de la radiación solar en Mega Julios por metro cuadrado (MJ/m²).

* Rainfall (Lluvia): Precipitación de lluvia en milímetros (mm).

* Snowfall (Nieve): Precipitación de nieve en centímetros (cm).

* Date (Fecha): Año-mes-día de la observación. Usada para seguimiento de tendencias.

* Hour (Hora): Hora del día (0 a 23). Muestra un patrón de demanda cíclico (horas pico de mañana y tarde).

* Seasons (Estaciones): Estación del año (Invierno, Primavera, Verano, Otoño). Variable categórica que influye significativamente en el uso (mayor demanda en Primavera/Verano).

* Holiday (Día Festivo): Indica si es un día festivo o no (Holiday/No holiday). Afecta los patrones de viaje.

* Functional Day (Día Funcional): Indica si las horas son funcionales para el servicio (Fun/NoFunc). Es clave, ya que en horas NoFunc la demanda será casi nula.

In [2]:
np.random.seed(1989)

In [3]:
pd.options.display.float_format = '{:,.3f}'.format

In [4]:
#Cargamos los datos
df = pd.read_csv(r"/content/SeoulBikeData.csv", encoding='latin1')
df.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.200,37,2.200,2000,-17.600,0.000,0.000,0.000,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.500,38,0.800,2000,-17.600,0.000,0.000,0.000,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.000,39,1.000,2000,-17.700,0.000,0.000,0.000,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.200,40,0.900,2000,-17.600,0.000,0.000,0.000,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.000,36,2.300,2000,-18.600,0.000,0.000,0.000,Winter,No Holiday,Yes


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       8760 non-null   object 
 1   Rented Bike Count          8760 non-null   int64  
 2   Hour                       8760 non-null   int64  
 3   Temperature(°C)            8760 non-null   float64
 4   Humidity(%)                8760 non-null   int64  
 5   Wind speed (m/s)           8760 non-null   float64
 6   Visibility (10m)           8760 non-null   int64  
 7   Dew point temperature(°C)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)    8760 non-null   float64
 9   Rainfall(mm)               8760 non-null   float64
 10  Snowfall (cm)              8760 non-null   float64
 11  Seasons                    8760 non-null   object 
 12  Holiday                    8760 non-null   object 
 13  Functioning Day            8760 non-null   objec

# Valores ausentes

In [6]:
def is_na_df(df):
    #Establecemos el resumen de NAs por columna
    na_summary = (
        df.isna().sum()
        .reset_index()
        .rename(columns={"index": "columna", 0: "n_missing"})
    )

    #Tomamos totales y prporciones
    na_summary["total"] = len(df)
    na_summary["prop_missing"] = na_summary["n_missing"] / na_summary["total"]
    na_summary["prop_original"] = 1 - na_summary["prop_missing"]

    #Filtramos solo columnas con al menos 1 NA
    na_summary = na_summary[na_summary["n_missing"] > 0].reset_index(drop=True)

    #Agregamos fila resumen global
    total_missing = df.isna().sum().sum()
    total_elements = df.size
    prop_missing_global = total_missing / total_elements
    prop_original_global = 1 - prop_missing_global

    resumen = pd.DataFrame({
        "columna": ["TOTAL"],
        "n_missing": [total_missing],
        "total": [total_elements],
        "prop_missing": [prop_missing_global],
        "prop_original": [prop_original_global]
    })

    na_summary = pd.concat([na_summary, resumen], ignore_index=True)

    return na_summary

In [7]:
is_na_df(df)

,columna,n_missing,total,prop_missing,prop_original
0,TOTAL,0,122640,0.000,1.000


No existen valores ausentes

In [9]:
# Definimos las variables numericas y categoricas
var_num = ["Rented Bike Count", "Hour", "Temperature(°C)", "Humidity(%)", "Wind speed (m/s)", "Visibility (10m)", "Dew point temperature(°C)", "Solar Radiation (MJ/m2)", "Rainfall(mm)", "Snowfall (cm)"]
var_cat = ["Date", "Seasons", "Holiday", "Functioning Day"]

In [10]:
#Transformamos las variables categorícas
for var in var_cat:
    df[var] = df[var].astype("category")

Graficas

In [11]:
#Definimos un objeto para graficar
class analyze:
    # Inicializamos la clase
    def __init__(self, df, num_vars, cat_vars):
        self.df = df
        self.num_vars = num_vars
        self.cat_vars = cat_vars

    #Método para graficar variables numéricas
    def plot_num(self):
        for var in self.num_vars:
            fig = make_subplots(rows=1, cols=2)

            # Boxplot
            fig.add_trace(
                go.Box(y=self.df[var], name="Boxplot", marker_color="#de3163"),
                row=1, col=1
            )

            # Histograma
            fig.add_trace(
                go.Histogram(x=self.df[var], name="Histograma",
                             marker_color="#31deac", nbinsx=50,
                             histnorm='probability density'),
                row=1, col=2
            )

            fig.update_layout(title_text=f"Variable: {var}", height=400, width=800)
            fig.show()

    #Método para graficar variables categóricas
    def plot_cat(self):
        for var in self.cat_vars:
            value_counts = self.df[var].value_counts()
            labels = value_counts.index.astype(str)
            values = value_counts.values

            fig = go.Figure(
                data=[go.Pie(labels=labels, values=values,textinfo='label+value+percent',
                    marker=dict(colors=["#de3163", "#31deac"])
                                )
                    ]
            )

            fig.update_layout(title_text=f"Variable: {var}", height=400, width=500)
            fig.show()

    #Método para graficar la matriz de correlación de numéricas
    def corr_matrix(self):
        corr_matrix = self.df[self.num_vars].corr()

        fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.columns,
            colorscale='Inferno'))

        fig.update_layout(
            title='Matriz de Correlación',
            width=800,
            height=600
        )

        fig.show()

In [12]:
#Definimos la Curva de Andrews escalada
def andrews_curve_scaled(df):
    df_aux = df.copy()

    #Escalamos solo las columnas numéricas
    numeric_cols = df_aux.select_dtypes(include=['float64', 'int64']).columns
    scaler = MinMaxScaler()
    df_aux[numeric_cols] = scaler.fit_transform(df_aux[numeric_cols])

    #Agregamos una categoría ficticia
    df_aux["dummy_class"] = ""

    plt.figure(figsize=(12,8))
    andrews_curves(df_aux, "dummy_class", colormap="viridis", alpha=0.2)
    plt.title("Curvas de Andrews")
    plt.xlabel("t (función de Fourier)")
    plt.ylabel("Valor transformado")
    plt.grid(True, color="lightgray", linestyle="--", linewidth=0.5)
    plt.show()

In [13]:
analisis = analyze(df, var_num, var_cat)

In [14]:
analisis.plot_num()

In [15]:
analisis.plot_cat()

In [16]:
analisis.corr_matrix()

#Valores atipicos

In [17]:
#Aplicamos Isolation forest para valores atípicos.
def detect_outliers_iforest(df, n_estimators=100, contamination=0.05, random_state=1989):
    model = IForest(n_estimators=n_estimators, contamination=contamination, random_state=random_state)
    model.fit(df)
    preds = model.predict(df)
    return df[preds == 1]

In [18]:
df_outliers = detect_outliers_iforest(df[var_num])
df_outliers

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning:

X has feature names, but IsolationForest was fitted without feature names



,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
141,165,21,1.100,92,0.500,235,0.000,0.000,0.700,1.000
143,193,23,1.300,92,0.600,209,0.100,0.000,0.000,1.000
144,198,0,1.400,92,0.500,240,0.200,0.000,0.000,0.900
145,218,1,1.500,92,0.400,227,0.300,0.000,0.000,0.900
146,149,2,1.400,92,0.700,168,0.200,0.000,0.000,0.900
...,...,...,...,...,...,...,...,...,...,...
8625,355,9,2.300,88,1.100,286,0.500,0.410,0.000,2.500
8626,430,10,4.000,76,1.300,404,0.100,0.930,0.000,2.400
8627,522,11,6.000,66,0.900,573,0.100,1.370,0.000,2.300
8628,700,12,6.900,59,0.900,669,-0.500,1.310,0.000,2.200


In [19]:
df_outliers.shape

(438, 10)

In [20]:
percentage_outliers = (len(df_outliers) / len(df)) * 100

print(f"El porcentaje de valores atípicos es: {percentage_outliers:.2f}%")

El porcentaje de valores atípicos es: 5.00%


Minimo porcentaje, procedemos a borrar los valores atípicos

In [ ]:
#Eliminamos los atipicos
df.drop(df_outliers.index, inplace=True)

In [21]:
df

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.200,37,2.200,2000,-17.600,0.000,0.000,0.000,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.500,38,0.800,2000,-17.600,0.000,0.000,0.000,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.000,39,1.000,2000,-17.700,0.000,0.000,0.000,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.200,40,0.900,2000,-17.600,0.000,0.000,0.000,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.000,36,2.300,2000,-18.600,0.000,0.000,0.000,Winter,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,30/11/2018,1003,19,4.200,34,2.600,1894,-10.300,0.000,0.000,0.000,Autumn,No Holiday,Yes
8756,30/11/2018,764,20,3.400,37,2.300,2000,-9.900,0.000,0.000,0.000,Autumn,No Holiday,Yes
8757,30/11/2018,694,21,2.600,39,0.300,1968,-9.900,0.000,0.000,0.000,Autumn,No Holiday,Yes
8758,30/11/2018,712,22,2.100,41,1.000,1859,-9.800,0.000,0.000,0.000,Autumn,No Holiday,Yes


In [22]:
df.shape

(8760, 14)

# Codificiación de variables

In [23]:
var_num

['Rented Bike Count',
 'Hour',
 'Temperature(°C)',
 'Humidity(%)',
 'Wind speed (m/s)',
 'Visibility (10m)',
 'Dew point temperature(°C)',
 'Solar Radiation (MJ/m2)',
 'Rainfall(mm)',
 'Snowfall (cm)']

In [24]:
var_cat

['Date', 'Seasons', 'Holiday', 'Functioning Day']

In [25]:
#DEFINIMOS NUESTRA Y QUE ES NUESTRA VARIABLE OBEJTIVO
y = 'Rented Bike Count'

In [26]:
y_col = 'Rented Bike Count'

# Variables explicativas (todas menos la y)
X = df.drop(columns=[y_col])
y = df[y_col]

In [27]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first',
                    handle_unknown='ignore',
                    sparse_output=False)   # matriz densa para armar DataFrame

X_cat_encoded = ohe.fit_transform(X[var_cat])

cat_cols = ohe.get_feature_names_out(var_cat)

X_cat_df = pd.DataFrame(X_cat_encoded,
                        columns=cat_cols,
                        index=X.index)

In [28]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

var_num = [x for x in var_num if x != 'Rented Bike Count']

X_num_scaled = scaler.fit_transform(X[var_num])

X_num_df = pd.DataFrame(X_num_scaled,
                        columns=var_num,
                        index=X.index)

In [30]:
X_processed = pd.concat([X_num_df, X_cat_df], axis=1)

display(X_processed.head())

print('\n{0}'.format(X_processed.shape))

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Date_01/02/2018,...,Date_31/05/2018,Date_31/07/2018,Date_31/08/2018,Date_31/10/2018,Date_31/12/2017,Seasons_Spring,Seasons_Summer,Seasons_Winter,Holiday_No Holiday,Functioning Day_Yes
0,0.000,0.220,0.378,0.297,1.000,0.225,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000
1,0.043,0.215,0.388,0.108,1.000,0.225,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000
2,0.087,0.206,0.398,0.135,1.000,0.223,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000
3,0.130,0.203,0.408,0.122,1.000,0.225,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000
4,0.174,0.206,0.367,0.311,1.000,0.208,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000



(8760, 378)


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_processed,
    y,
    test_size=0.3,
    random_state=7725
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6132, 378), (2628, 378), (6132,), (2628,))

# Modelado

In [32]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

def regression_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, mape, r2

In [33]:
# Espacio de búsqueda de hiperparámetros
param_dist = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': np.logspace(-3, 3, 15),
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 2, 6)),
    'degree': [2, 3, 4, 5],
    'coef0': np.linspace(0, 1, 5)
}


In [37]:
# Modelo base
svr = SVR()

In [38]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=svr,
    param_distributions=param_dist,
    n_iter=10,
    scoring='neg_root_mean_squared_error',
    cv=5,
    random_state=1989,
    n_jobs=-1,
    verbose=1
)
random_search.fit(X_train, y_train)
best_svr = random_search.best_estimator_

print("\nMejores parámetros encontrados:")
print(random_search.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits

Mejores parámetros encontrados:
{'kernel': 'linear', 'gamma': np.float64(100.0), 'degree': 2, 'coef0': np.float64(1.0), 'C': np.float64(7.196856730011514)}


In [40]:
y_pred_train = best_svr.predict(X_train)
y_pred_test = best_svr.predict(X_test)

# Métricas
rmse_train, mae_train, mape_train, r2_train = regression_metrics(y_train, y_pred_train)
rmse_test, mae_test, mape_test, r2_test = regression_metrics(y_test, y_pred_test)

In [41]:
results = pd.DataFrame({
    'Set': ['Entrenamiento', 'Prueba'],
    'RMSE': [rmse_train, rmse_test],
    'MAE': [mae_train, mae_test],
    'MAPE (%)': [mape_train, mape_test]
})

display(results)

print("\n✅ Entrenamiento y evaluación completados.")

,Set,RMSE,MAE,MAPE (%)
0,Entrenamiento,446.414,304.473,"3,206,113,428,281,458,688.000"
1,Prueba,466.508,318.333,"3,460,187,468,025,237,504.000"



✅ Entrenamiento y evaluación completados.
